# 2.1 Dataset Artigos SciELO

## Imports

In [11]:
import os

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, SoupStrainer
from tqdm import tqdm

## Código

In [2]:
path_artigos = os.path.join(os.getcwd(), 'artigos_brutos', 'SciELO')

In [3]:
# contagem de artigos
soma = len(os.listdir(path_artigos))
print(f'Total de artigos: {soma}')

Total de artigos: 1681


### Criando o DataSet

In [4]:
dict_df = {'title': [], 'seção': [], 'ano': [],  'doi_id': [], 'text_html': []}

for arquivo in tqdm(os.listdir(path_artigos)):
    path_abs  = os.path.join(path_artigos, arquivo)
    
    with open(path_abs, 'r', encoding='utf8') as arq:
        texto_html = arq.read()
        
    soup = BeautifulSoup(texto_html, 'lxml')
    
    try:
        doi_id = soup.find('a', attrs={'class': '_doi'}).text.strip().removeprefix('https://doi.org/')
    except AttributeError:
        doi_id = ''

    try:
        titulo = ' '.join(soup.find('h1', attrs={'class': 'article-title'}).text.split())
    except AttributeError:
        titulo = ''

    try:
        seção = soup.find('span', attrs={'class': '_articleBadge'}).text.strip().lower()
    except AttributeError:
        seção = ''

    try:
        ano = soup.find('span', attrs={'class': '_editionMeta'}).text.split('•')[-1].strip()
        if len(ano) > 4:
            ano = ano[-4:]
    except AttributeError:
        ano = ''

    dict_df['title'].append(titulo)
    dict_df['doi_id'].append(doi_id)
    dict_df['text_html'].append(texto_html)
    dict_df['seção'].append(seção)
    dict_df['ano'].append(int(ano))
        

100%|██████████| 1681/1681 [02:29<00:00, 11.25it/s]


In [14]:
# salvando o dataset em feather
df = pd.DataFrame(dict_df)
df.to_feather('scielo_dataset.feather')